In [9]:
import numpy as np

In [1]:
### KNIGHT ALLOWED MOVES (OFFSETS)
allowedOffsetMoves = {(-2,-1),
                      (-1,-2),
                      (+1,-2),
                      (+2,-1),
                      (+2,+1),
                      (+1,+2),
                      (-1,+2),
                      (-2,+1)
}

### SIX BASE STRUCTURED KNIGHT TOURS
## Coords based on chess algebraic notation

####### 6x6 #######
##    # # # # # # 6
##    # # # # # # 5
##    # # # # # # 4
##    + # # # # # 3
##    # # # # # # 2
##    # # # # # # 1
##    1 2 3 4 5 6
##
##    Cell + is (1,3)

path6x6 = {
    (1,1): (2,3),
    (2,3): (3,1),
    (3,1): (1,2),
    (1,2): (3,3),
    (3,3): (5,2),
    (5,2): (6,4),
    (6,4): (5,6),
    (5,6): (3,5),
    (3,5): (1,6),
    (1,6): (2,4),
    (2,4): (4,3),
    (4,3): (5,5),
    (5,5): (3,6),
    (3,6): (1,5),
    (1,5): (3,4),
    (3,4): (2,6),
    (2,6): (1,4),
    (1,4): (2,2),
    (2,2): (4,1),
    (4,1): (6,2),
    (6,2): (5,4),
    (5,4): (6,6),
    (6,6): (4,5),
    (4,5): (5,3),
    (5,3): (6,1),
    (6,1): (4,2),
    (4,2): (2,1),
    (2,1): (1,3),
    (1,3): (2,5),
    (2,5): (4,6),
    (4,6): (6,5),
    (6,5): (4,4),
    (4,4): (6,3),
    (6,3): (5,1),
    (5,1): (3,2),
    (3,2): (1,1)
}

In [119]:
class Chessboard:
    
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns
        self.path = {}
        
    def SetPath(self, path):
        self.path = path.copy()
    
    def GetPath(self):
        return self.path.copy()
    
    def GetRows(self):
        return self.rows
    
    def GetColumns(self):
        return self.columns
    
    def FindPath(self):
        if((self.GetRows() == 6) and (self.GetColumns() == 6)):
            self.SetPath(path6x6)
            return
        
        nRows = self.GetRows()
        nColumns = self.GetColumns()
        
        topLeftBoard = Chessboard(nRows/2, nColumns/2)
        topRightBoard = Chessboard(nRows/2, nColumns/2)
        bottomLeftBoard = Chessboard(nRows/2, nColumns/2)
        bottomRightBoard = Chessboard(nRows/2, nColumns/2)
        
        topLeftBoard.FindPath()
        topRightBoard.FindPath()
        bottomLeftBoard.FindPath()
        bottomRightBoard.FindPath()
        
        ## Build path from bottom left, makes sense for coordinates
        bottomRightPath = bottomRightBoard.GetPath()
        topLeftPath = topLeftBoard.GetPath()
        topRightPath = topRightBoard.GetPath()
        
        newBottomLeftPath = bottomLeftBoard.GetPath()
        newBottomRightPath = {}
        newTopLeftPath = {}
        newTopRightPath = {}
        
        for position in bottomRightPath:
            newPosition = tuple(map(lambda i, j: int(i + j), position, (nColumns/2,0)))
            newNextStep = tuple(map(lambda i, j: int(i + j), bottomRightPath[position], (nColumns/2,0)))
            newBottomRightPath[newPosition] = newNextStep
        
        for position in topLeftPath:
            newPosition = tuple(map(lambda i, j: int(i + j), position, (0,nRows/2)))
            newNextStep = tuple(map(lambda i, j: int(i + j), topLeftPath[position], (0,nRows/2)))
            newTopLeftPath[newPosition] = newNextStep

        ## This one needs to be inverted for topology: bottom left corner always runs opposite to the others
        for position in topRightPath:
            newPosition = tuple(map(lambda i, j: int(i + j), position, (nColumns/2,nRows/2)))
            newNextStep = tuple(map(lambda i, j: int(i + j), topRightPath[position], (nColumns/2,nRows/2)))
            #newTopRightPath[newPosition] = newNextStep
            newTopRightPath[newNextStep] = newPosition
        
        ## Fix the edges
        newBottomLeftPath[(nColumns/2-1,nRows/2-2)] = (nColumns/2+1,nRows/2-1)
        newBottomRightPath[(nColumns/2+3,nRows/2)] = (nColumns/2+1,nRows/2+1)
        newTopRightPath[(nColumns/2+2,nRows/2+3)] = (nColumns/2,nRows/2+2)
        newTopLeftPath[(nColumns/2-2,nRows/2+1)] = (nColumns/2,nRows/2)
        
        newCompletePath = {**newBottomLeftPath, **newBottomRightPath, **newTopLeftPath, **newTopRightPath}
        self.SetPath(newCompletePath)
        return
    
    def PrintPath(self):
        nextStep = self.GetPath()
        startingPosition = (1,1)
        currentPosition = nextStep[startingPosition]
        
        tourMatrix = np.zeros((self.GetRows(),self.GetColumns()),int)
        
        visitedPositions = {startingPosition: 1}
        tourMatrix[self.GetRows()-startingPosition[1],startingPosition[0]-1] = 1
        positionCounter = 1
        
        while currentPosition != startingPosition:
            positionCounter += 1
            
            if currentPosition in visitedPositions:
                print("ERROR: position", currentPosition, "was already visited before returning to", startingPosition, ".")
                print("It was visited as square", visitedPositions[currentPosition], "and would be again as square", positionCounter,".")
                break
            
            #print(self.GetRows()-currentPosition[1], currentPosition[0]-1)
            tourMatrix[int(self.GetRows()-currentPosition[1]),int(currentPosition[0]-1)] = positionCounter
            visitedPositions[currentPosition] = positionCounter
            currentPosition = nextStep[currentPosition]
        
        if len(visitedPositions) != self.GetRows() * self.GetColumns():
            print("ERROR: ", len(visitedPositions), "visited positions, but", self.GetRows() * self.GetColumns(), "squares on the board.")
        
        print(tourMatrix)
    
    ## Check if path covers every square exactly once without breaking
    def PathIsATour(self):
        nextStep = self.GetPath()
        startingPosition = (1,1)
        currentPosition = nextStep[startingPosition]
        
        visitedPositions = {startingPosition: True}
        
        while currentPosition != startingPosition:
            if currentPosition in visitedPositions:
                print("ERROR: position", currentPosition, "was already visited before returning to", startingPosition, ".")
                return False
            
            visitedPositions[currentPosition] = True
            currentPosition = nextStep[currentPosition]
        
        if len(visitedPositions) != self.GetRows() * self.GetColumns():
            print("ERROR: ", len(visitedPositions), "visited positions, but", self.GetRows() * self.GetColumns(), "squares on the board.")
            return False
        
        return True
    
    ## Check if all moves in the path can be performed by a knight
    def PathIsLegal(self):
        isLegal = True
        nextStep = self.GetPath()
        
        for position in nextStep:
            isLegal *= (tuple(map(lambda i, j: i - j, nextStep[position], position)) in allowedOffsetMoves)
        
        return bool(isLegal)
        
    
    ## Check if path is structured
    #def PathIsStructured(self):
    
    ## Check if path is closed
    #def PathIsClosed(self):
    
    ## Combine all previous checks
    #def PathIsAClosedStructuredTour(self):

In [123]:
board = Chessboard(24,24)
#board.SetPath(path6x6)
board.GetPath()

{}

In [124]:
print(board.GetRows())
print(board.GetColumns())

24
24


In [125]:
board.FindPath()
print(board.GetPath())
print(board.PathIsATour())
print(board.PathIsLegal())
print(board.PrintPath())

{(1, 1): (2, 3), (2, 3): (3, 1), (3, 1): (1, 2), (1, 2): (3, 3), (3, 3): (5, 2), (5, 2): (6, 4), (6, 4): (5, 6), (5, 6): (3, 5), (3, 5): (1, 6), (1, 6): (2, 4), (2, 4): (4, 3), (4, 3): (5, 5), (5, 5): (3, 6), (3, 6): (1, 5), (1, 5): (3, 4), (3, 4): (2, 6), (2, 6): (1, 4), (1, 4): (2, 2), (2, 2): (4, 1), (4, 1): (6, 2), (6, 2): (5, 4), (5, 4): (7.0, 5.0), (6, 6): (4, 5), (4, 5): (5, 3), (5, 3): (6, 1), (6, 1): (4, 2), (4, 2): (2, 1), (2, 1): (1, 3), (1, 3): (2, 5), (2, 5): (4, 6), (4, 6): (6, 5), (6, 5): (4, 4), (4, 4): (6, 3), (6, 3): (5, 1), (5, 1): (3, 2), (3, 2): (1, 1), (7, 1): (8, 3), (8, 3): (9, 1), (9, 1): (7, 2), (7, 2): (9, 3), (9, 3): (11, 2), (11, 2): (12, 4), (12, 4): (11, 6), (11, 6): (9, 5), (9, 5): (7, 6), (7, 6): (8, 4), (8, 4): (10, 3), (10, 3): (11, 5), (11, 5): (9, 6), (9, 6): (7.0, 7.0), (7, 5): (9, 4), (9, 4): (8, 6), (8, 6): (7, 4), (7, 4): (8, 2), (8, 2): (10, 1), (10, 1): (12, 2), (12, 2): (11, 4), (11, 4): (12, 6), (12, 6): (10, 5), (10, 5): (11, 3), (11, 3): (

In [6]:
print((4,1), "->", path6x6[(4,1)])
print(path6x6[(4,1)], "->", path6x6[path6x6[(4,1)]])
print()
print((1,1), "->", path6x6[(1,1)])
print(path6x6[(1,1)], "->", path6x6[path6x6[(1,1)]])
print()
print((5,4), "->", path6x6[(5,4)])
print(path6x6[(5,4)], "->", path6x6[path6x6[(5,4)]])
print()
print((3,6), "->", path6x6[(3,6)])
print(path6x6[(3,6)], "->", path6x6[path6x6[(3,6)]])

(4, 1) -> (6, 2)
(6, 2) -> (5, 4)

(1, 1) -> (2, 3)
(2, 3) -> (3, 1)

(5, 4) -> (6, 6)
(6, 6) -> (4, 5)

(3, 6) -> (1, 5)
(1, 5) -> (3, 4)


In [10]:
## Next steps:
# 1. Implementa i check mancanti
# 2. Aggiusta l'algoritmo perché funzioni anche con quadrati più grossi (sempre a potenze di 6)

In [91]:
nrow = 4
ncol = 5
mat = np.zeros((nrow,ncol),int)
pos = (2,3)
posTranslated = (nrow-pos[1],pos[0]-1)
mat[posTranslated] = 1
#mat[0,1] = 1
print(posTranslated)
print(mat)

(1, 1)
[[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [53]:
mat = np.zeros((nrow,ncol),int)
mat[1,3] = 1
mat

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])